In [ ]:
!pip install pathway bokeh --quiet


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
import pandas as pd


df1 = pd.read_csv("dataset.csv")
df1

In [ ]:
df1['Timestamp'] = pd.to_datetime(df1['LastUpdatedDate'] + ' ' + df1['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

In [ ]:
df1 = df1.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# df1.to_csv("parking_stream_1.csv", index=False)
df1[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream1.csv", index=False)


In [ ]:
# class ParkingSchema1(pw.Schema):
#     ID: int
#     SystemCodeNumber: str
#     Capacity: int
#     Latitude: float
#     Longitude: float
#     Occupancy: int
#     VehicleType: str
#     TrafficConditionNearby: str
#     QueueLength: int
#     IsSpecialDay: int
#     LastUpdatedDate: str
#     LastUpdatedTime: str
#     Timestamp: str  # already parsed into full timestamp
class ParkingSchema1(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location

In [ ]:
data1 = pw.demo.replay_csv("parking_stream1.csv", schema=ParkingSchema1, input_rate=1000)


In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time1 = data1.with_columns(
    t1 = data1.Timestamp.dt.strptime(fmt),
    day1 = data1.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
import datetime

delta_window1 = (
    data_with_time1.windowby(
        pw.this.t1,
        instance=pw.this.day1,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t1=pw.this._pw_window_end,
        occ_max1=pw.reducers.max(pw.this.Occupancy),
        occ_min1=pw.reducers.min(pw.this.Occupancy),
        cap1=pw.reducers.max(pw.this.Capacity),
        #queue_avg=pw.reducers.mean(pw.this.QueueLength),  # optional
    )
    .with_columns(
        price1=10 + (pw.this.occ_max1 - pw.this.occ_min1) / pw.this.cap1
    )
)


In [ ]:
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t1", "price1", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t1", "price1", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window1.plot(price_plotter, sorting_col="t1")
# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import seaborn as sns

In [ ]:
df2 = pd.read_csv('dataset.csv')
df2

In [ ]:
 #Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df2['Timestamp'] = pd.to_datetime(df2['LastUpdatedDate'] + ' ' + df2['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index

traffic_map = {'low': 1, 'average': 2, 'high': 3}
vehicle_map = {'car': 1.0, 'bike': 0.5,'truck': 2.0, 'cycle': 0.2}  # Ex
df2['traffic_level_num'] =  df2["TrafficConditionNearby"].map(traffic_map).fillna(0)
df2['vehicle_weight_num'] =  df2["VehicleType"].map(vehicle_map).fillna(0)
df2 = df2.sort_values('Timestamp').reset_index(drop=True)
df2

In [ ]:
df2[["Timestamp", "Occupancy", "Capacity","QueueLength","traffic_level_num","IsSpecialDay","vehicle_weight_num"]].to_csv("parking_stream2.csv", index=False)


In [ ]:
park=pd.read_csv("parking_stream2.csv")
park

In [ ]:
sns.heatmap(park.isnull())

In [ ]:
class ParkingSchema2(pw.Schema):
    #ID: int
   # SystemCodeNumber: str
    Capacity: int
   # Latitude: float
   # Longitude: float
    Occupancy: int
    vehicle_weight_num: float
    traffic_level_num: int
    QueueLength: int
    IsSpecialDay: int
   # LastUpdatedDate: str
   # LastUpdatedTime: str
    Timestamp: str  # already parsed into full timestamp




In [ ]:
data2 = pw.demo.replay_csv("parking_stream2.csv", schema=ParkingSchema2, input_rate=1000)


In [ ]:
ALPHA = 0.6   # Weight for occupancy rate
BETA = 0.2    # Weight for queue length
GAMMA = 0.15  # Penalty for traffic level
DELTA = 0.3   # Boost for special day
EPSILON = 0.1 # Vehicle type weight
LAMBDA = 0.5  # Price adjustment factor
BASE_PRICE = 10.0

# Define mappings for categorical features
# traffic_map = {'low': 1, 'average': 2, 'high': 3}
# vehicle_map = {'car': 1.0, 'bike': 0.5,'truck': 2.0, 'cycle': 0.2}  # Extend this as needed

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time2 = data2.with_columns(
    t2 = data2.Timestamp.dt.strptime(fmt),
    day2 = data2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),


)

In [ ]:
import datetime

delta_window2 = (
    data_with_time2.windowby(
        pw.this.t2,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day2,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    ) .reduce(
        t2=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max2=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min2=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap2=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)

        queue_length_num = pw.reducers.max(pw.this.QueueLength), # Include queue length
        traffic_level_num = pw.reducers.max(pw.this.traffic_level_num), # Include traffic level
        is_special_day_num = pw.reducers.max(pw.this.IsSpecialDay), # Include special day
        vehicle_weight_num = pw.reducers.max(pw.this.vehicle_weight_num) # Include vehicle weight
    ))


In [ ]:
reduced_data=delta_window2.with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6
        demand= (ALPHA*((pw.this.occ_max2 - pw.this.occ_min2) / pw.this.cap2)
                        +BETA*pw.this.queue_length_num
                        -GAMMA*pw.this.traffic_level_num
                        +DELTA*pw.this.is_special_day_num
                        +EPSILON*pw.this.vehicle_weight_num)

    )


In [ ]:
norm_red_data=reduced_data.with_columns(
    normalized_demand = pw.this.demand / (1 + abs(pw.this.demand))
)



In [ ]:
price_data= norm_red_data.with_columns(
        price2=10 * (1 + LAMBDA*pw.this.normalized_demand)
)

In [ ]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 2: Demand-Based Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t2", "price2", source=source, line_width=2, color="darkblue", legend_label="Dynamic Price")
    fig.circle("t2", "price2", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

# Create reactive visualization
viz = price_data.plot(price_plotter, sorting_col="t2")

# Display it as a Panel app
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()